In [4]:
import numpy as np
import pandas as pd
import pickle
#from build_database import flux_obj
from scipy import interpolate
import matplotlib.pyplot as plt
import os
import itertools
import random
#%matplotlib inline
# Autoload changes made in external editor:
%load_ext autoreload
%autoreload 2

# --------------- Latex Plot Beautification --------------------------
fig_width_pt = 650.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = fig_width*golden_mean      # height in inches
fig_size =  [fig_width+1,fig_height+1]
params = {'backend': 'ps',
          'axes.labelsize': 14,
          'text.fontsize': 14,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': False,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
# --------------- Latex Plot Beautification --------------------------

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from calc_scattering import calc_scattering
import time

directory='/Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/python/'
center_lat=45
lower_freq=200
upper_freq=240
L_shells=2.5
I0 = -100000
dlat = 1


tstart = time.time()
ray_crossings = calc_scattering(directory, I0, center_lat, lower_freq, upper_freq, L_shells, dlat)
tstop = time.time()

print "Elapsed time: %g seconds"%(tstop - tstart)

loading  /Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/python/newray200.dat
loading  /Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/python/newray240.dat
Ray starting at 31 degrees
testing 1107 cases (coarse grid)
Ray starting at 32 degrees
testing 1105 cases (coarse grid)
Ray starting at 33 degrees
testing 1078 cases (coarse grid)
Ray starting at 34 degrees
testing 1030 cases (coarse grid)
Ray starting at 35 degrees
testing 969 cases (coarse grid)
Ray starting at 36 degrees
testing 899 cases (coarse grid)
Ray starting at 37 degrees
testing 818 cases (coarse grid)
Ray starting at 38 degrees
testing 734 cases (coarse grid)
Ray starting at 39 degrees
testing 649 cases (coarse grid)
Ray starting at 40 degrees
testing 563 cases (coarse grid)
Ray starting at 41 degrees
testing 483 cases (coarse grid)
Ray starting at 42 degrees
testing 404 cases (coarse grid)
Ray starting at 43 degrees
testing 335 cases (coarse grid)
Ray starting at 44 degrees
testing 266 cases (coarse grid)
Elapsed ti

In [7]:
# Plot the equal-area slices, along with fieldlines, to show that the coordinates are right.
from calc_scattering import gen_EA_array
from matplotlib import collections  as mc
import sim_consts as sc

L_targ = L_shells
EA = gen_EA_array(L_targ)

# Initialize plot
fig, ax = plt.subplots()

# Plot the earth
earth = plt.Circle((0,0),1,color='k',alpha=0.3)

# Plot the fieldline
lam = np.linspace(-50,50,181)
L_r = L_targ*pow(np.cos(lam*sc.D2R),2)
Lx  = L_r*np.cos(lam*sc.D2R)
Ly  = L_r*np.sin(lam*sc.D2R)

# Plot the margin fieldlines
Lmr1 = (L_targ + sc.L_MARGIN)*pow(np.cos(lam*sc.D2R),2)
Lmr1x= Lmr1*np.cos(lam*sc.D2R)
Lmr1y= Lmr1*np.sin(lam*sc.D2R)
Lmr2 = (L_targ - sc.L_MARGIN)*pow(np.cos(lam*sc.D2R),2)
Lmr2x= Lmr2*np.cos(lam*sc.D2R)
Lmr2y= Lmr2*np.sin(lam*sc.D2R)

# Plot Equal-Area slices
po = zip(EA['x1'],EA['y1'])
p1 = zip(EA['x2'],EA['y2'])
points = zip(po, p1)
lc = mc.LineCollection(points)

# Plot Ray Crossing Segments
rayc = mc.LineCollection(ray_crossings[1].values(),colors=(0,0.5,0.5,1))


plt.plot(Lx,Ly,color='r',linewidth=2)  # Field line
plt.plot(Lmr1x, Lmr1y,color='g')       # Outer boundary
plt.plot(Lmr2x, Lmr2y,color='g')       # Inner boundary
ax.add_patch(earth)                    # The earth (round)
ax.add_collection(lc)                  # Equal-Area segments
ax.add_collection(rayc)                # Crossed ray segments

#ax.autoscale()
ax.set_xlim([2.3,2.6])
ax.set_ylim([-0.5,0.5])
ax.set_xlabel('L')
ax.set_ylabel('L')



plt.show()

TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
tmp = ray_crossings[0]
tmp.values()
print np.shape(tmp.values())

print tmp.values()


#ray_segments = mc.LineCollection(tmp.values(),colors=(1,0,0,1))



po_ray = []
p1_ray = []
for row in tmp.values():
    po_ray.append(row[0])
    p1_ray.append(row[1])




In [ ]:
print ray_crossings

In [ ]:
from load_rayfile import load_rayfile
rf = load_rayfile(directory,200)

tb = rf[35].ELE > 2.5
#print rf[35].iloc[1]
#print rf[35][tb]
x = rf[35][tb]
y = np.linspace(1,50,50)
tmp = np.flat(np.outer(x,y))
print np.shape(tmp)

In [ ]:
import sim_consts as sc

#EA_NUMROWS = (sc.EALimN - sc.EALimS) / sc.EAIncr + 1
#EA_NUMCOLS = 1 + 8*NUM_TARGS;

lam_vec = np.arange(sc.EALimS,sc.EALimN, step=sc.EAIncr)
L_TARG = 2.5

# Trig functions
clam = np.cos(lam_vec*sc.D2R)
slam = np.sin(lam_vec*sc.D2R)
clam2= pow(clam,2)
slam2= pow(slam,2)
rootTerm = np.sqrt(1 + 3*slam2)

# dl_lam: still not sure where he gets this expression from
dl_lam = (clam2*clam / rootTerm)*sc.DL0

x_unit_vect = (3*clam2 - 2) / rootTerm
y_unit_vect = (3*slam*clam) / rootTerm


ptR = L_TARG*clam2  # Distance to field line (dipole model: r ~ L cos(latitude)^2 )
ptX = ptR*clam      # Field line in Cartesian coordinates
ptY = ptR*slam

# Coordinates of the edges of the window
x1 = ptX - x_unit_vect*dl_lam
x2 = ptX + x_unit_vect*dl_lam
y1 = ptY - y_unit_vect*dl_lam
y2 = ptY + y_unit_vect*dl_lam

# Output parameters:
d = pd.DataFrame(index=lam_vec,columns=['x1','x2','y1','y2','EAa','EAb','EAc','EA_length'])
d['EAa'] = y1 - y2
d['EAb'] = x2 - x1
d['EAc'] = x1*y2 - y1*x2
d['EA_length'] = np.sqrt(pow(x1-x2,2)+pow(y1-y2,2))*sc.R_E
d['x1'] = x1
d['x2'] = x2
d['y1'] = y1
d['y2'] = y2

#print d
d['EA_length'].plot()
